In [3]:
# Importing necessary libraries
import sqlite3
import pandas as pd

# Create a SQLite database
conn = sqlite3.connect('weather.db')
cur = conn.cursor()

# Creating the Weather table
cur.execute('DROP TABLE IF EXISTS Weather')
cur.execute('''
    CREATE TABLE Weather (
        id INTEGER PRIMARY KEY,
        recordDate DATE,
        temperature INTEGER
    )
''')

# Inserting data into the Weather table
data = [
    (1, '2015-01-01', 10),
    (2, '2015-01-02', 25),
    (3, '2015-01-03', 20),
    (4, '2015-01-04', 30)
]
cur.executemany('INSERT INTO Weather (id, recordDate, temperature) VALUES (?, ?, ?)', data)

# Commit changes and close connection
conn.commit()

# SQL Query to find IDs with higher temperatures than the previous day
query = '''
    SELECT w1.id 
    FROM Weather w1
    JOIN Weather w2 ON w1.recordDate = DATE(w2.recordDate, '+1 day')
    WHERE w1.temperature > w2.temperature
'''

# Executing the  SQL query
cur.execute(query)
sql_results = cur.fetchall()

# Printing results from SQL
print("SQL Result:")
sql_df = pd.DataFrame(sql_results, columns=['id'])
print(sql_df)

# Using pandas for the same logic
# Creating a DataFrame from the original data
df = pd.DataFrame(data, columns=['id', 'recordDate', 'temperature'])

# Sorting DataFrame by date
df['recordDate'] = pd.to_datetime(df['recordDate'])
df = df.sort_values('recordDate')

# Finding IDs with higher temperatures than previous dayy
df['prev_temp'] = df['temperature'].shift(1)  # Adding previous day's temp
result_df = df[df['temperature'] > df['prev_temp']]

# Print results from Python

print("\nPython Result:")
print(result_df[['id']])

# Closing the database connection
conn.close()


SQL Result:
   id
0   2
1   4

Python Result:
   id
1   2
3   4
